# FloML - Example Analysis

This notebook demonstrates the key capabilities of the FloML (Flood Machine Learning) package:

1. **Segmented Linear Regression** - Model non-linear stage-discharge relationships
2. **Multi-Station Correlation** - Analyze upstream-downstream timing
3. **Precursor Detection** - Identify early flood warning signals

Data is read from the PostgreSQL database curated by the Rust monitoring daemon.

In [ ]:
import sys
sys.path.insert(0, '..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from floml.db import get_engine, verify_schemas
from floml.regression import fit_segmented_regression, fit_stage_discharge
from floml.correlation import correlate_stations, find_optimal_lag
from floml.precursors import analyze_precursors, calculate_rise_rate

# Set plotting style
sns.set_style('whitegrid')
%matplotlib inline

## 1. Connect to Database

In [ ]:
engine = get_engine()
verify_schemas(engine)
print("✓ Database connected")

## 2. Load Available Stations

In [ ]:
stations = pd.read_sql("""
    SELECT site_code, site_name, latitude, longitude, monitored
    FROM usgs_raw.sites
    ORDER BY site_code
""", engine)

stations

## 3. Segmented Regression - Stage-Discharge Relationship

Rivers often have non-linear stage-discharge curves due to:
- Channel overflow into floodplain
- Backwater effects
- Ice jams

We use piecewise linear fitting to model these regime changes.

In [ ]:
# Load paired stage-discharge data for Kingston Mines
site_code = '05568500'

data = pd.read_sql(f"""
    SELECT reading_time,
           MAX(CASE WHEN parameter_code = '00065' THEN value END) as stage_ft,
           MAX(CASE WHEN parameter_code = '00060' THEN value END) as discharge_cfs
    FROM usgs_raw.gauge_readings
    WHERE site_code = '{site_code}'
      AND reading_time > NOW() - INTERVAL '1 year'
    GROUP BY reading_time
    HAVING MAX(CASE WHEN parameter_code = '00065' THEN value END) IS NOT NULL
       AND MAX(CASE WHEN parameter_code = '00060' THEN value END) IS NOT NULL
""", engine)

print(f"Loaded {len(data)} paired observations")
data.head()

In [ ]:
if len(data) >= 50:
    # Fit 3-segment model
    result = fit_stage_discharge(data['discharge_cfs'], data['stage_ft'], n_segments=3)
    
    print(result)
    
    # Plot
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Scatter plot
    ax.scatter(data['discharge_cfs'], data['stage_ft'], alpha=0.5, s=10, label='Observed')
    
    # Fitted line
    x_range = np.linspace(data['discharge_cfs'].min(), data['discharge_cfs'].max(), 500)
    y_pred = result.predict(x_range)
    ax.plot(x_range, y_pred, 'r-', linewidth=2, label='Fitted (3 segments)')
    
    # Mark breakpoints
    for bp in result.breakpoints[1:-1]:
        ax.axvline(bp, color='green', linestyle='--', alpha=0.5)
    
    ax.set_xlabel('Discharge (cfs)')
    ax.set_ylabel('Stage (ft)')
    ax.set_title(f'Stage-Discharge Relationship: {site_code}\nR² = {result.r_squared:.4f}')
    ax.legend()
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
else:
    print("Not enough data for regression analysis")

## 4. Multi-Station Correlation

Analyze how upstream stations correlate with downstream, including timing lag.

In [ ]:
# Load stage data for two stations (if available)
upstream_code = '05568500'  # Kingston Mines
downstream_code = '05570000'  # Peoria

# Note: You may need to adjust these site codes based on what's in your database
print(f"Analyzing correlation: {upstream_code} → {downstream_code}")

In [ ]:
upstream_data = pd.read_sql(f"""
    SELECT reading_time as timestamp, value as stage_ft
    FROM usgs_raw.gauge_readings
    WHERE site_code = '{upstream_code}'
      AND parameter_code = '00065'
      AND reading_time > NOW() - INTERVAL '90 days'
    ORDER BY reading_time
""", engine)

downstream_data = pd.read_sql(f"""
    SELECT reading_time as timestamp, value as stage_ft
    FROM usgs_raw.gauge_readings
    WHERE site_code = '{downstream_code}'
      AND parameter_code = '00065'
      AND reading_time > NOW() - INTERVAL '90 days'
    ORDER BY reading_time
""", engine)

print(f"Upstream: {len(upstream_data)} readings")
print(f"Downstream: {len(downstream_data)} readings")

In [ ]:
if len(upstream_data) > 100 and len(downstream_data) > 100:
    upstream_series = upstream_data.set_index('timestamp')['stage_ft']
    downstream_series = downstream_data.set_index('timestamp')['stage_ft']
    
    # Find correlation
    corr_result = correlate_stations(upstream_series, downstream_series)
    print(corr_result)
else:
    print("Insufficient data for correlation analysis")

## 5. Flood Precursor Detection

Analyze a historical flood event to identify early warning signals.

In [ ]:
# Load a flood event
events = pd.read_sql("""
    SELECT e.id, e.site_code, e.crest_time, e.peak_stage_ft, e.severity
    FROM nws.flood_events e
    WHERE e.crest_time IS NOT NULL
    ORDER BY e.crest_time DESC
    LIMIT 5
""", engine)

events

In [ ]:
if len(events) > 0:
    # Analyze first event
    event = events.iloc[0]
    site_code = event['site_code']
    crest_time = pd.Timestamp(event['crest_time'])
    
    # Load stage data 14 days before peak
    window_start = crest_time - pd.Timedelta(days=14)
    
    stage_data = pd.read_sql(f"""
        SELECT reading_time, value as stage_ft
        FROM usgs_raw.gauge_readings
        WHERE site_code = '{site_code}'
          AND parameter_code = '00065'
          AND reading_time BETWEEN '{window_start}' AND '{crest_time}'
        ORDER BY reading_time
    """, engine)
    
    if len(stage_data) > 0:
        stage_series = stage_data.set_index('reading_time')['stage_ft']
        
        # Detect precursors
        precursors = analyze_precursors(stage_series, crest_time)
        
        print(f"\nEvent: {site_code} on {crest_time}")
        print(f"Peak: {event['peak_stage_ft']:.2f} ft ({event['severity']})")
        print(f"\nFound {len(precursors)} precursor events:")
        
        for p in precursors:
            print(f"  • {p.precursor_type:15s} {p.hours_before_peak:6.1f}h before peak")
        
        # Plot
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), sharex=True)
        
        # Stage hydrograph
        ax1.plot(stage_series.index, stage_series.values, 'b-', linewidth=1)
        ax1.axvline(crest_time, color='red', linestyle='--', label='Peak')
        ax1.axhline(event['peak_stage_ft'], color='red', linestyle=':', alpha=0.5)
        
        # Mark precursors
        for p in precursors:
            ax1.axvline(p.detected_at, color='orange', alpha=0.3)
        
        ax1.set_ylabel('Stage (ft)')
        ax1.set_title(f'Flood Event Analysis: {site_code}')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        
        # Rise rate
        rise_rate = calculate_rise_rate(stage_series)
        ax2.plot(rise_rate.index, rise_rate.values, 'g-', linewidth=1)
        ax2.axhline(0, color='black', linestyle='-', linewidth=0.5)
        ax2.axhline(0.5, color='orange', linestyle='--', alpha=0.5, label='Rapid rise threshold')
        ax2.set_ylabel('Rise Rate (ft/day)')
        ax2.set_xlabel('Time')
        ax2.legend()
        ax2.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
    else:
        print("No stage data available for this event")
else:
    print("No flood events found in database")
    print("Run: cargo run --bin ingest_peak_flows")

## Next Steps

1. **More Data**: Ingest historical data using the Rust daemon
   ```bash
   cd ../flomon_service
   cargo run --bin historical_ingest
   cargo run --bin ingest_peak_flows
   ```

2. **Custom Analysis**: Use FloML modules for your specific analysis

3. **Write Results**: Store analysis results back to `flood_analysis` schema